### Imports

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, cv2
from google.colab import drive


### #Find an alternative approach to opening/uploading img file

In [66]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
image_folder = '/content/drive/MyDrive/Unsplash-imgs'

## Assign Lighting Score

In [68]:
def calc_weighted_sum(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l = lab[:, :, 0].astype(np.float32)/255
    perception = l.mean()
    shadow     = (l < 0.15).mean() #the avg l-values of the number of pixels w/l value < 15%
    highlight  = (l > 0.85).mean() #the avg l-values of the number of pixels w/l value > 85%
    score = perception - shadow + highlight*0.15 #-1(dark) ---- 1(bright)
    return perception, shadow, highlight, score

print("Lighting score (P - (S+H)): ", calc_weighted_sum(img))


Lighting score (P - (S+H)):  (np.float32(0.6877265), np.float64(0.00020645112090714784), np.float64(0.05353273752152739), np.float64(0.6955499571574685))


### Try to make the score more accurate of the lighting and easier to work with

## Next: Create CSV file to include all images and their *scores*

In [69]:
import csv
data = [] #to-be a dictionary containing info to store in csv
def write_dict(filename):
      img = cv2.imread(os.path.join(image_folder, filename)) #reads the image from filename
      if img is not None:
        size = img.shape #(px, px, color channel)
        perception, shadow, highlight, score = calc_weighted_sum(img)
        data.append(
            {'Filename': filename, 'Size': size, 'Perception': perception, 'Shadow': shadow, 'Highlight': highlight, 'Score': score}
        )
      else:
        print(f"Warning: Could not read image file: {filename}")
      return data
for filename in os.listdir(image_folder): #iterates through each file in the drive folder
  write_dict(filename)
print(data)

[{'Filename': 'very-bright-img.jpg', 'Size': (6240, 4160, 3), 'Perception': np.float32(0.85878026), 'Shadow': np.float64(0.0), 'Highlight': np.float64(0.6084724790433925), 'Score': np.float64(0.95005113671094)}, {'Filename': 'bright-sky-img.jpg', 'Size': (1333, 2000, 3), 'Perception': np.float32(0.88188916), 'Shadow': np.float64(0.006882970742685671), 'Highlight': np.float64(0.7876219054763691), 'Score': np.float64(0.9931494795265542)}, {'Filename': 'night-sky-img.jpg', 'Size': (3376, 6000, 3), 'Perception': np.float32(0.00751835), 'Shadow': np.float64(0.9958293345181675), 'Highlight': np.float64(0.0006495359399684045), 'Score': np.float64(-0.9882135539804613)}, {'Filename': 'beach-img.jpg', 'Size': (3967, 5950, 3), 'Perception': np.float32(0.6877265), 'Shadow': np.float64(0.00020645112090714784), 'Highlight': np.float64(0.05353273752152739), 'Score': np.float64(0.6955499571574685)}, {'Filename': 'marie-michele-bouchard-bEZQWH49daU-unsplash.jpg', 'Size': (5509, 4000, 3), 'Perception': 

In [70]:
with open('image-lightings.csv', 'w', newline='') as csvfile: #new csv file
    fieldnames = ['Filename', 'Size', 'Perception', 'Shadow', 'Highlight', 'Score'] #columns
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

df = pd.read_csv('image-lightings.csv')
print(df.to_string()) #prints the csv file

                                           Filename             Size  Perception    Shadow  Highlight     Score
0                               very-bright-img.jpg  (6240, 4160, 3)    0.858780  0.000000   0.608472  0.950051
1                                bright-sky-img.jpg  (1333, 2000, 3)    0.881889  0.006883   0.787622  0.993149
2                                 night-sky-img.jpg  (3376, 6000, 3)    0.007518  0.995829   0.000650 -0.988214
3                                     beach-img.jpg  (3967, 5950, 3)    0.687727  0.000206   0.053533  0.695550
4   marie-michele-bouchard-bEZQWH49daU-unsplash.jpg  (5509, 4000, 3)    0.617449  0.108828   0.317692  0.556274
5               omid-armin-6G2G6_rq-B0-unsplash.jpg  (6000, 4000, 3)    0.195693  0.532827   0.000986 -0.336986
6            tarikul-raana-YiG7BqusLc0-unsplash.jpg  (6000, 4000, 3)    0.458507  0.116763   0.000398  0.341803
7           behrouz-sasani-1B22UBPK1Pw-unsplash.jpg  (2500, 2000, 3)    0.641290  0.013777   0.055756  0